In [1]:
#check the direction

In [2]:
import os
print(os.getcwd())

C:\Program Files\Microsoft VS Code


In [3]:
import os

# Change to your project directory (update the path as needed)
os.chdir(r"H:\Work_With_MZ\LC_Classification_Code")

# Verify the change
print(os.getcwd())

H:\Work_With_MZ\LC_Classification_Code


In [4]:
import xarray as xr
import pandas as pd
import pandas as gpd
import regionmask
import numpy as np
import geopandas
import xarray as xr
import xarray as xr
import seaborn as sns
import rioxarray
from shapely.geometry import mapping
import netCDF4
import netCDF4 as nc
from PIL import Image
import os
from netCDF4 import Dataset
import geopandas as gpd
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from scipy.stats import linregress
#from xarrayMannKendall import MannKendall
import pymannkendall as mk
import xarray as xr
import numpy as np
from scipy.stats import kendalltau

In [9]:
import os
import rasterio
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling
import geopandas as gpd
import numpy as np
import xarray as xr

# Paths
tif_folder = "tif_files"
bands = ["B2.tif", "B3.tif", "B4.tif", "B8.tif", "B11.tif", "B12.tif"]
shapefile = "Province_Sofian/Export_Output.shp"

# Reference band (for resolution, transform, crs)
ref_path = os.path.join(tif_folder, "B2.tif")
with rasterio.open(ref_path) as ref:
    ref_crs = ref.crs
    ref_transform = ref.transform
    ref_shape = (ref.height, ref.width)
    ref_profile = ref.profile

# Read and reproject shapefile to raster CRS
gdf = gpd.read_file(shapefile)
if gdf.crs != ref_crs:
    gdf = gdf.to_crs(ref_crs)
roi_geom = [gdf.geometry.unary_union]

# Prepare output arrays
band_arrays = []
band_names = []

for band in bands:
    band_path = os.path.join(tif_folder, band)
    with rasterio.open(band_path) as src:
        # Read and resample to reference grid if needed
        data = src.read(1)
        if (src.transform != ref_transform) or (src.shape != ref_shape) or (src.crs != ref_crs):
            data_resampled = np.empty(ref_shape, dtype=data.dtype)
            reproject(
                source=data,
                destination=data_resampled,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=ref_transform,
                dst_crs=ref_crs,
                resampling=Resampling.bilinear
            )
        else:
            data_resampled = data

        # Mask (clip) with ROI
        # To use rasterio.mask, we need a dataset object, so use MemoryFile
        with rasterio.io.MemoryFile() as memfile:
            with memfile.open(
                driver='GTiff',
                height=ref_shape[0],
                width=ref_shape[1],
                count=1,
                dtype=data_resampled.dtype,
                crs=ref_crs,
                transform=ref_transform,
            ) as dataset:
                dataset.write(data_resampled, 1)
                out_image, out_transform = mask(
                    dataset,
                    roi_geom,
                    crop=True,
                    filled=True,
                    nodata=0,
                    all_touched=True
                )
        band_arrays.append(out_image[0])
        band_names.append(band.split('.')[0])

# Stack and save as NetCDF
stacked = np.stack(band_arrays, axis=0)  # shape: (bands, y, x)
da = xr.DataArray(
    stacked,
    dims=("band", "y", "x"),
    coords={"band": band_names}
)
ds = xr.Dataset({"sentinel2": da})
ds.to_netcdf("sentinel2_clipped.nc")
print("Saved as sentinel2_clipped.nc")

Saved as sentinel2_clipped.nc


In [ ]:
import xarray as xr
import numpy as np
import geopandas as gpd
import rasterio
import rasterio.features
from shapely.geometry import shape
from rasterio.features import rasterize
from sklearn.metrics import confusion_matrix

# 1. Load NetCDF and extract bands
ds = xr.open_dataset("sentinel2_clipped.nc")
B3 = ds['sentinel2'].sel(band='B3').values.astype('float32')
B8 = ds['sentinel2'].sel(band='B8').values.astype('float32')

# 2. Calculate NDWI and threshold (handle division by zero)
ndwi = np.where((B3 + B8) == 0, np.nan, (B3 - B8) / (B3 + B8))
water_mask = (ndwi > 0.2) & ~np.isnan(ndwi)

# 3. Get transform and CRS from original raster
with rasterio.open("tif_files/B2.tif") as src:
    crs = src.crs
    transform = src.transform

# 4. Convert water mask to polygons and set CRS
shapes = rasterio.features.shapes(water_mask.astype('uint8'), transform=transform)
water_polys = [shape(geom) for geom, value in shapes if value == 1]
water_gdf = gpd.GeoDataFrame(geometry=water_polys, crs=crs)  # Set CRS here

# 5. Reproject water polygons to WGS84
water_gdf = water_gdf.to_crs("EPSG:4326")

# 6. Load and reproject observation shapefile to WGS84
obs_gdf = gpd.read_file("Pools_Observation/Pools.shp")
obs_gdf = obs_gdf.to_crs("EPSG:4326")

# 7. Polygon-wise validation
joined = gpd.sjoin(water_gdf, obs_gdf, how='left', predicate='intersects')
TP = len(joined[~joined['index_right'].isna()])
FP = len(water_gdf) - TP
FN = len(obs_gdf) - TP
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print("Polygon-wise metrics:")
print(f"  True Positives: {TP}")
print(f"  False Positives: {FP}")
print(f"  False Negatives: {FN}")
print(f"  Precision: {precision:.2f}")
print(f"  Recall: {recall:.2f}")
print(f"  F1 Score: {f1:.2f}")

# 8. Pixel-wise validation (confusion matrix)
# Rasterize observed polygons to match water_mask grid (in original CRS)
obs_gdf_pix = obs_gdf.to_crs(crs)
obs_raster = rasterize(
    [(geom, 1) for geom in obs_gdf_pix.geometry],
    out_shape=water_mask.shape,
    transform=transform,
    fill=0,
    dtype='uint8'
)
y_true = obs_raster.flatten()
y_pred = water_mask.astype('uint8').flatten()
cm = confusion_matrix(y_true, y_pred)
TP_pix = cm[1,1]
FP_pix = cm[0,1]
FN_pix = cm[1,0]
TN_pix = cm[0,0]
precision_pix = TP_pix / (TP_pix + FP_pix) if (TP_pix + FP_pix) > 0 else 0
recall_pix = TP_pix / (TP_pix + FN_pix) if (TP_pix + FN_pix) > 0 else 0
f1_pix = 2 * precision_pix * recall_pix / (precision_pix + recall_pix) if (precision_pix + recall_pix) > 0 else 0

print("\nPixel-wise metrics:")
print(f"  Confusion Matrix:\n{cm}")
print(f"  Precision: {precision_pix:.2f}")
print(f"  Recall: {recall_pix:.2f}")
print(f"  F1 Score: {f1_pix:.2f}")

# 9. Save results as shapefiles in WGS84
water_gdf.to_file("detected_water_bodies_WGS84.shp")
obs_gdf.to_file("observed_pools_WGS84.shp")